In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import numpy as np
import pickle
from tabulate import tabulate
from functools import partial
import ctypes
from scipy.optimize import minimize, LinearConstraint, dual_annealing, Bounds, NonlinearConstraint
from scipy.stats import truncnorm
import scipy.stats as stats
from scipy.stats.qmc import Sobol, Halton
import matplotlib.pyplot as plt
import matplotlib as mpl
import nlopt 
import multiprocessing, tqdm
from bgspy.likelihood import negll_c, negll, negll_numba
from bgspy.likelihood import bounds_simplex, random_start_simplex, random_start_mutation
from bgspy.likelihood import negll_mutation, bounds_mutation
from bgspy.likelihood import penalized_negll_c
from bgspy.likelihood import normal_ll_c

from bgspy.optim import run_optims

In [3]:
def nloptim_summary(run_res, nt, nf, mle_rank=0, nll=None):
    nlls, thetas, success = run_res
    ranks = np.argsort(nlls)
    best_idx = ranks[mle_rank]
    theta = thetas[best_idx]
    # if we're a parameter short, assume μ is fixed
    dim = theta.size
    fixedmu = dim == nt * nf + 1
    print(fixedmu, dim)
    nll = nlls[best_idx]
    status, counts = np.unique(success, return_counts=True)
    n = len(success)
    stat_codes = {1:'success', 2:'stopval reached', 3:'ftol reached', 
                  4:'xtol reached', 5:'max eval', 6:'maxtime reached', -1:'failure',
                  -2:'invalid args', -3:'out of memory', -4:'forced stop'}
    print("exist status stats: " + ','.join([f"{stat_codes[i]} = {np.round(c/n, 2)*100}%" for 
                                             i, c in zip(status, counts)]))
    print(f"exists: ")
    print(f"NLL: {nll}\npi0: {theta[0]}\n")
    if not fixedmu:
        print(f"μ: {theta[1]}\nW = ")
        o = 2
    else:
        print(f"μ: --fixed?--\nW = ")
        o = 1
    W = theta[o:].reshape((nt, nf))
    print(tabulate(np.round(W, 3)))
    print(f"sums: {W.sum(axis=0)}")
    return theta

def summarize_optims(res, nt, nf, mle_rank=0):
    # for scipy
    nlls, thetas, success = res
    ranks = np.argsort(nlls)
    best_idx = ranks[mle_rank]
    x = thetas[best_idx]
    ll = nlls[best_idx]
    pi0 = x[0]
    W = x[1:].reshape((nt, nf))
    mu = W.sum(axis=0)
    print(f"convergence: {np.round(np.mean(success), 2)*100}%")
    print(f"neg loglik = {ll}\nμ = {mu}\nπ0 = {pi0}\nw = \n")
    print(tabulate(W / W.sum(axis=0)))
    return x

def check_bounds(x, lb, ub):
    assert np.all((x >= lb) & (x <= ub))
    

def run_nloptims(workerfunc, startfunc, nstarts, ncores = 70):
    starts = [startfunc() for _ in range(nstarts)]
    with multiprocessing.Pool(ncores) as p:
        res = list(tqdm.tqdm(p.imap(workerfunc, starts), total=nstarts))
    nlls = np.array([x[0] for x in res])
    thetas = np.array([x[1] for x in res])
    success = np.array([x[2] for x in res])
    return nlls, thetas, success

In [4]:
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 200
plt.style.use('./buffalo.mplstyle')

# Fitting and Testing Optimization Procedures

Note that we often have to define specific fitting functions and run those — annoyingly when using Python's multiprocessing module, there's an issue where closures (functions that define functions around values) do not pickle and thus cannot be shipped off to other cores safely. So each test relies on a custom test function.

## Load data and define some functions

In [5]:
dat = pickle.load(open('../tests/likelihood_test_data.pkl', 'rb'))
B, Bp, Y, w, t = dat['B'], dat['Bp'], dat['Y'], dat['w'], dat['t']
bins = dat['bins']

nx, nw, nt, nf = B.shape
nparams = nt * nf + 2

### EDA of the Count Data / π

In [6]:
def optim_plot(tail=0.5, **runs):
    fig, ax = plt.subplots()
    for i, (key, run) in enumerate(runs.items()):
        nll = run[0]
        succ = run[2]
        q = np.quantile(nll, tail)
        nlls = nll[nll < q]
        sort_idx = np.argsort(nlls)
        y = nlls[sort_idx]
        x = 2*i + succ[nll < q].astype('int')
        cols = mpl.cm.get_cmap('Paired')(x)
        ax.scatter(list(reversed(range(len(y)))), y, s=1, label=key, c=cols)
    ax.legend()

In [7]:
assert Bp.shape[0] == Y.shape[0]
assert B.shape[0] == Y.shape[0]

# Free-Mutation Model (SciPy)

Here we try a less-constrained form of optimization: a separate mutation rate per feature/selection class. This minimizes constraints — all we need are bounds. But, there are an extra two degrees of freedom. The simpler constraints mean that SciPy's optimiation function will work fine.

## Free-Mutation Model with Reduced Features Model

Here we try the simplest test case: only one column of features (CDS) of the test human data.
 

In [8]:
def negll_freemut(Y, B, w):
    def func(theta):
        new_theta = np.full(theta.size + 1, np.nan)
        theta = np.copy(theta)
        new_theta[0] = theta[0]
        # fix mutation rate to one and let W represent mutation rates to various classes
        new_theta[1] = 1.
        new_theta[2:] = theta[1:] # times mutation rates
        #print("-->", theta, new_theta)
        return negll_c(new_theta, Y, B, w)
    return func

In [9]:
w

array([1.00000000e-11, 6.30957344e-11, 3.98107171e-10, 2.51188643e-09,
       1.58489319e-08, 1.00000000e-07])

In [10]:
Y.shape, B.shape

((3044, 2), (3044, 6, 5, 3))

In [11]:
from bgspy.likelihood import LikelihoodFreeMutation

In [12]:
nllBp = negll_freemut(Y, B, w)
func = partial(minimize, nllBp)

In [ ]:
lik = LikelihoodFreeMutation(w, t, B, Y=Y)

lik.fit(n=1_000, ncores=70)

 12%|████████████████████████████▌                                                                                                                                                                                                           | 123/1000 [00:59<05:04,  2.88it/s]

In [19]:
lik.optim

OptimResult
  success: True (termination code: success)
  negative log-likelihood = 1026498389842.0181
  theta = [1.47369165e-03 1.00000000e-11 1.00000000e-11 1.00000000e-11
 1.00000000e-11 1.00000000e-11 1.00000000e-11 1.00000000e-11
 1.00000000e-11 1.00000000e-11 1.00000000e-11 1.00000000e-11
 7.92380657e-09 1.00000000e-11 9.52796822e-10 3.47017676e-10]

In [ ]:
lik.fit_multicore()

In [22]:
nllBp = negll_freemut(Y, Bp[..., 0][..., None], w)
nllB = negll_freemut(Y, B[..., 0][..., None], w)

from bgspy.optim import scipy_worker

def worker_Bp_reduced(start):
    return minimize(nllBp, x0=start, bounds=bounds_mutation(nt, 1, paired=True), method='L-BFGS-B')

def worker_B_reduced(start):
    return minimize(nllB, x0=start, bounds=bounds_mutation(nt, 1, paired=True), method='L-BFGS-B')

#resBp_reduced = run_optims(worker_Bp_reduced, partial(random_start_mutation, nt, 1), 100)
#resB_reduced = run_optims(worker_B_reduced, partial(random_start_mutation, nt, 1), 100)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.69it/s]


In [24]:
resB_reduced = run_optims(partial(minimize, fun=nllBp, bounds=bounds_mutation(nt, 1, paired=True), method='L-BFGS-B'), partial(random_start_mutation, nt, 1), 1)

  0%|                                                                                                                                                                                                                                                     | 0/1 [00:00<?, ?it/s]


AttributeError: Can't pickle local object 'negll_freemut.<locals>.func'

In [21]:
summarize_optims(resB_reduced, nt, 1)
summarize_optims(resBp_reduced, nt, 1)

NameError: name 'summarize_optims' is not defined

In [ ]:
optim_plot(B=resB_reduced, Bp=resBp_reduced, tail=0.7)
plt.semilogy()

## Free-Mutation Model (All Features)

First we try with SciPy's `minimize()` function:

In [ ]:
nllBp = negll_freemut(Y, Bp, w)
nllB = negll_freemut(Y, B, w)

def worker_Bp_full(start):
    return minimize(nllBp, x0=start, bounds=bounds_mutation(nt, nf, paired=True), options={'eps':1e-9}, method='L-BFGS-B')

def worker_B_full(start):
    return minimize(nllB, x0=start, bounds=bounds_mutation(nt, nf, paired=True), method='L-BFGS-B')

resB = run_optims(worker_Bp_full, partial(random_start_mutation, nt, nf), 10_000)
resBp = run_optims(worker_Bp_full, partial(random_start_mutation, nt, nf), 10_000)

In [ ]:
summarize_optims(resB, nt, nf)
summarize_optims(resBp, nt, nf)

Now let's try with NLOpt's optimization approaches.

In [ ]:
def negll_freemut_nlopt(Y, B, w):
    def func(x, grad):
        new_theta = np.zeros(x.size + 1)
        new_theta[0] = x[0]
        # fix mutation rate to one and let W represent mutation rates to various classes
        new_theta[1] = 1.
        new_theta[2:] = x[1:] # times mutation rates        
        return negll_c(new_theta, Y, B, w)
    return func

In [ ]:
nparams = nt*nf+1
opt = nlopt.opt(nlopt.GN_ISRES, nparams)
nllBp = negll_freemut_nlopt(Y, Bp, w)

# set the objective
opt.set_min_objective(nllBp) 

# set bounds
lb, ub = bounds_mutation(nt, nf)
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)

In [ ]:
%%time
#np.random.seed(0)
# stopping criteria:
opt.set_xtol_rel(1e-5)
opt.set_maxeval(10000000)

# random start
x = random_start_mutation(nt, nf)
assert x.size == nparams

# run
res = opt.optimize(x)

In [ ]:
opt

In [ ]:
e = res[1:].reshape((nt, nf))
res[0], e.sum(axis=0), e/e.sum(axis=1)[:, None], opt.last_optimum_value()

# Simplex Model (NLOpt)

The simplex model is an $n_t \times n_f$ matrix $\mathbf{W}$ which are DFE weights (columns sum to 1). The mutation rate per selection class, per feature class is then $\mu \mathbf{W}$.

The bounds of $\mathbf{W}$ are $[0, 1]$, and the constraint is $\sum_j w_{i,j} = 1$ However, there is also a total mutation bound, $\mu_l \le \mu \le \mu_u$. So, 

$$ \mu_l \le \mu \mathbf{W} \le \mu_u$$

In [ ]:
def negll_nlopt(Y, B, w):
    # NLOPT needs to accept a grad argument
    def func(x, grad):
        return negll_c(x, Y, B, w)
    return func

In [ ]:
def constraint_matrix(nt, nf):
    nparams = nt*nf + 2
    A = np.zeros((nf, nparams))
    for i in range(nf):
        W = A[i, 2:].reshape((nt, nf))
        W[:, i] = 1.
    return A

def inequality_constraint_functions(nt, nf, log10_mu_bounds=(-11, -7)):
    """
     l < μW < u
     l - μW < 0
     μW - u < 0
    """
    A = constraint_matrix(nt, nf)
    lower, upper = 10**log10_mu_bounds[0], 10**log10_mu_bounds[1]
    def func_l(result, x, grad):
        mu = x[1]
        M = lower - (mu *  A.dot(x))
        for i in range(nf):
            result[i] = M[i]
    def func_u(result, x, grad):
        mu = x[1]
        M = (mu *  A.dot(x)) - upper
        for i in range(nf):
            result[i] = M[i]
    return func_l, func_u

def equality_constraint_function(nt, nf):
    A = constraint_matrix(nt, nf)
    def func(result, x, grad):
        M = A.dot(x)
        for i in range(nf):
            result[i] = M[i] - 1.
    return func

## A simple NLOpt single run test

In [ ]:
Y.shape, Bp.shape

In [ ]:
hl, hu = inequality_constraint_functions(nt, nf)
ce = equality_constraint_function(nt, nf)

# NOTE: we do not have a start-based test of these bounds
# because it looks like SWIG's interface to NLOPT wraps this in a 
# weird way

In [ ]:
nparams = nt * nf + 2 # 1 for mutation rate, one for π0
# specify the algorithm
#opt = nlopt.opt(nlopt.LN_COBYLA, nparams)
opt = nlopt.opt(nlopt.GN_ISRES, nparams)

# create a closure around the data
nll = negll_nlopt(Y, Bp, w)

# set the objective
opt.set_min_objective(nll) 

# specify tolerances and constraints
tols = np.repeat(1e-11, nf)
opt.add_inequality_mconstraint(hl, tols)
opt.add_inequality_mconstraint(hu, tols)
opt.add_equality_mconstraint(ce, tols)

# set bounds
lb, ub = bounds_simplex(nt, nf)
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)

In [ ]:
%%time
np.random.seed(1231)
# stopping criteria:
#opt.set_xtol_abs(np.array([1e-4, 1e-10] + [1e-04] * nt*nf))
opt.set_xtol_rel(1e-3)
#opt.set_ftol_abs(1e8)
#opt.set_stopval(923543002497)
opt.set_maxeval(10000000)

# random start
x = random_start_simplex(nt, nf)
assert x.size == nparams

# run
res = opt.optimize(x)

In [ ]:
# mle theta, nll:
res, opt.last_optimum_value(), opt.last_optimize_result()

In [ ]:
res[0], res[1], np.round(res[2:].reshape((nt, nf)), 2), opt.last_optimum_value()

In [ ]:
res[2:].reshape((nt, nf)).sum(axis=0)

In [ ]:
opt.get_initial_step(x)

## Exploratory ISRES Global Runs

In [ ]:
# this wraps the nloptimization call
def nlopt_isres_worker(x, nt, nf):
    nparams = nt*nf + 2
    opt = nlopt.opt(nlopt.GN_ISRES, nparams)
    #opt = nlopt.opt(nlopt.AUGLAG, nparams)
    #opt.set_local_optimizer(nlopt.LN_COBYLA)
    nll = negll_nlopt(Y, Bp, w)
    opt.set_min_objective(nll) 
    hl, hu = inequality_constraint_functions(nt, nf)
    tols = np.repeat(1e-11, nf)
    opt.add_inequality_mconstraint(hl, tols)
    opt.add_inequality_mconstraint(hu, tols)
    ce = equality_constraint_function(nt, nf)
    opt.add_equality_mconstraint(ce, tols)
    lb, ub = bounds_simplex(nt, nf)
    opt.set_lower_bounds(lb)
    opt.set_upper_bounds(ub)
    opt.set_xtol_rel(1e-3)
    #opt.set_xtol_abs(1e-6)
    #opt.set_stopval(923543002497)
    opt.set_maxeval(1000000)
    assert x.size == nparams
    mle = opt.optimize(x)
    nll = opt.last_optimum_value()
    success = opt.last_optimize_result()
    return nll, mle, success

In [ ]:
%%time
N = 200

np.random.seed(0)
worker = partial(nlopt_isres_worker, nt=nt, nf=nf)
isres_run_res = run_nloptims(worker, partial(random_start_simplex, nt=nt, nf=nf), N)

In [ ]:
best_theta = nloptim_summary(isres_run_res, nt, nf) # isres run

Now, let's compare the performance of the free mutation model with random starts and local optimization to the ISRES simplex model:

In [ ]:
optim_plot(isres=isres_run_res, B=resB_reduced, Bp=resBp_reduced, tail=0.4)
plt.semilogy()

In [ ]:
isres_run_res[0].max()

## Free-Mutation Model (All Features, Gaussian LL)

In [ ]:
def normal_negll_freemut(Y, B, w):
    def func(theta):
        new_theta = np.zeros(theta.size + 1)
        new_theta[0] = theta[0]
        # fix mutation rate to one and let W represent mutation rates to various classes
        new_theta[1] = 1.
        new_theta[2:] = theta[1:] # times mutation rates        
        return -normal_ll_c(new_theta, Y, B, w)
    return func

In [ ]:
nllBp = normal_negll_freemut(Y, Bp, w)
nllB = normal_negll_freemut(Y, B, w)

def worker_Bp_full(start):
    return minimize(nllBp, x0=start, bounds=bounds_mutation(nt, nf, paired=True), options={'eps':1e-9}, method='L-BFGS-B')

def worker_B_full(start):
    return minimize(nllB, x0=start, bounds=bounds_mutation(nt, nf, paired=True), method='L-BFGS-B')

resB = run_optims(worker_Bp_full, partial(random_start_mutation, nt, nf), 100_000)
resBp = run_optims(worker_Bp_full, partial(random_start_mutation, nt, nf), 100_000)

In [ ]:
plt.scatter(np.sum((resBp[1][np.argmin(resBp[0])][None, :] - resBp[1])**2, axis=1), resBp[0])

In [ ]:
cutoff = resBp[0] < np.quantile(resBp[0], 0.01)
_ = plt.hist(resBp[1][cutoff, 2], 10)

In [ ]:
optim_plot(Bp=resBp, tail=0.01)

In [ ]:
optim_plot(B=resB, Bp=resBp, tail=0.1)
plt.semilogy()

## Free Mutation Model with MVN

In [ ]:
from scipy.sparse import lil_matrix

def random_start_mvn(nt, nf):
    theta = random_start_mutation(nt, nf)
    new_theta = np.empty(nt*nf + 3)
    new_theta[0] = theta[0]
    new_theta[1] = np.abs(np.random.normal(0, 1, 1))
    new_theta[2] = stats.invgamma(5).rvs(1)
    new_theta[3:] = theta[1:]
    return new_theta
    

def mvn_nll(Y, B, w, bins):
    TINY = 1e-8
    nx, nw, nt, nf = B.shape
    n = Y.shape[0]
    nS = Y[:, 0]
    nD = Y[:, 1]
    # reparameterized data under MVN approximation
    N = nS + nD

    def func(theta): 
        Sigma = np.zeros((n, n), dtype=float)
        pi0 = theta[0]
        alpha = theta[1]
        rho = theta[2]
        W = theta[3:].reshape((nt, nf))
        
        logBw = np.zeros(nx, dtype=float)
        for i in range(nx):
            for j in range(nt):
                for k in range(nf):
                    logBw[i] += np.interp(W[j, k], w, B[i, :, j, k])

        log_pibar = np.log(pi0) + logBw
        pi = np.exp(log_pibar)
        mean = N * pi
        var = N * (1-pi) * pi
    
        for i in range(n):
            for j in range(i, n):
                if bins[i][0] == bins[j][0]:
                    # same chromosome
                    Sigma[i, j] = np.sqrt(N[i] * N[j])*alpha**2  * np.exp(-0.5 / rho**2 * (i - j)**2) + var[i]*(i==j) + TINY
                    Sigma[j, i] = Sigma[i, j]
        return -stats.multivariate_normal.logpdf(x=nD, mean=mean, cov=Sigma)
    return func

def mvn_bounds(nt, nf):
    lb, ub = bounds_mutation(nt, nf)
    lb = lb.tolist()
    ub = ub.tolist()
    lb.insert(1, 0) # alpha
    ub.insert(1, np.Inf) # alpha
    lb.insert(2, 0) # rho
    ub.insert(2, np.Inf) # rho
    return list(zip(lb, ub))

In [ ]:
mvnnll = mvn_nll(Y, Bp, w, bins)

In [ ]:
mvnnll(random_start_mvn(nt, nf))

In [ ]:
len(mvn_bounds(nt, nf)), len(random_start_mvn(nt, nf))

In [ ]:
minimize(mvnnll, x0=random_start_mvn(nt, nf), bounds=mvn_bounds(nt, nf))

In [ ]:
a = _

## Large Exploratory Multistart Local Runs

In [ ]:

# this wraps the nloptimization call
def nlopt_worker(x, nt, nf):
    nparams = nt*nf + 2
    opt = nlopt.opt(nlopt.LN_COBYLA, nparams)
    #opt = nlopt.opt(nlopt.AUGLAG, nparams)
    #opt.set_local_optimizer(nlopt.LN_COBYLA)
    nll = negll_nlopt(Y, Bp, w)
    opt.set_min_objective(nll) 
    hl, hu = inequality_constraint_functions(nt, nf)
    tols = np.repeat(1e-11, nf)
    opt.add_inequality_mconstraint(hl, tols)
    opt.add_inequality_mconstraint(hu, tols)
    ce = equality_constraint_function(nt, nf)
    opt.add_equality_mconstraint(ce, tols)
    lb, ub = bounds_simplex(nt, nf)
    opt.set_lower_bounds(lb)
    opt.set_upper_bounds(ub)
    #opt.set_xtol_abs(np.array([1e-4, 1e-17] + [1e-6] * nt*nf))
    opt.set_xtol_rel(1e-1)
    #opt.set_ftol_abs(1e-8)
    opt.set_maxeval(100000)
    assert x.size == nparams
    mle = opt.optimize(x)
    nll = opt.last_optimum_value()
    success = opt.last_optimize_result()
    return nll, mle, success

In [ ]:
%%time
N = 1_000_000

#def best(theta):
#    return theta

np.random.seed(1)
worker = partial(nlopt_worker, nt=nt, nf=nf) 
#global_run_res = run_nloptims(worker, partial(best, theta=best_theta), N)
global_run_res = run_nloptims(worker, partial(random_start_simplex, nt=nt, nf=nf), N)

In [ ]:
optim_plot(global_run=global_run_res, tail=0.01)

In [ ]:
best_theta = nloptim_summary(global_run_res, nt, nf, 0)

## Fixing Mutation Rate

In [ ]:
constraint_matrix_fixedmu(nt, nf)[1, 1:].reshape((nt, nf))

In [ ]:
def constraint_matrix_fixedmu(nt, nf):
    nparams = nt*nf + 1
    A = np.zeros((nf, nparams))
    for i in range(nf):
        W = A[i, 1:].reshape((nt, nf))
        W[:, i] = 1.
    return A

def inequality_constraint_functions_fixedmu(nt, nf, mu):
    """
     l < μ W < u
     l - μ W < 0
     μ W - u < 0
    """
    A = constraint_matrix_fixedmu(nt, nf)
    lower = 0
    upper = 1
    def func_l(result, x, grad):
        M = lower - (mu *  A.dot(x))
        for i in range(nf):
            result[i] = M[i]
    def func_u(result, x, grad):
        M = (mu *  A.dot(x)) - upper
        for i in range(nf):
            result[i] = M[i]
    return func_l, func_u

def equality_constraint_function_fixedmu(nt, nf):
    A = constraint_matrix_fixedmu(nt, nf)
    def func(result, x, grad):
        M = A.dot(x)
        for i in range(nf):
            result[i] = M[i] - 1.
    return func


def negll_fixedmu_nlopt(Y, B, w, mu):
    # NLOPT needs to accept a grad argument
    def func(x, grad):
        theta = np.empty(x.size + 1)
        theta[0] = x[0]
        theta[1] = mu
        theta[2:] = x[1:].flat
        return negll_c(theta, Y, B, w)
    return func

def bounds_simplex_fixedmu(nt, nf, log10_pi0_bounds=(-4, -2),
           paired=False):
    l = [10**log10_pi0_bounds[0]]
    u = [10**log10_pi0_bounds[1]]
    l += [0.]*nf*nt
    u += [1.]*nf*nt
    lb = np.array(l)
    ub = np.array(u)
    assert np.all(lb < ub)
    if paired:
        return list(zip(lb, ub))
    return lb, ub


def nlopt_fixedmu_worker(x, nt, nf, mu):
    nparams = nt*nf + 1
    opt = nlopt.opt(nlopt.LN_COBYLA, nparams)
    #opt = nlopt.opt(nlopt.AUGLAG, nparams)
    #opt.set_local_optimizer(nlopt.LN_COBYLA)
    nll = negll_fixedmu_nlopt(Y, Bp, w, mu)
    opt.set_min_objective(nll) 
    hl, hu = inequality_constraint_functions_fixedmu(nt, nf, mu)
    tols = np.repeat(1e-11, nf)
    opt.add_inequality_mconstraint(hl, tols)
    opt.add_inequality_mconstraint(hu, tols)
    ce = equality_constraint_function_fixedmu(nt, nf)
    opt.add_equality_mconstraint(ce, tols)
    lb, ub = bounds_simplex_fixedmu(nt, nf)
    opt.set_lower_bounds(lb)
    opt.set_upper_bounds(ub)
    #opt.set_xtol_abs(np.array([1e-4, 1e-17] + [1e-6] * nt*nf))
    opt.set_xtol_rel(1e-1)
    #opt.set_ftol_abs(1e-8)
    opt.set_maxeval(100000)
    assert x.size == nparams
    mle = opt.optimize(x)
    nll = opt.last_optimum_value()
    success = opt.last_optimize_result()
    return nll, mle, success

def random_start_simplex_fixedmu(nt, nf, log10_pi0_bounds=(-4, -3)):
    pi0 = 10**np.random.uniform(log10_pi0_bounds[0], log10_pi0_bounds[1], 1)
    W = np.empty((nt, nf))
    for i in range(nf):
        W[:, i] = np.random.dirichlet([1.] * nt)
        assert np.abs(W[:, i].sum() - 1.) < 1e-5
    theta = np.empty(nt*nf + 1)
    theta[0] = pi0
    theta[1:] = W.flat
    return theta

In [ ]:
%%time
N = 10_000

np.random.seed(1)
worker = partial(nlopt_fixedmu_worker, nt=nt, nf=nf, mu=1e-8) 
fixedmu_run_res = run_nloptims(worker, partial(random_start_simplex_fixedmu, nt=nt, nf=nf), N)

In [ ]:
nloptim_summary(fixedmu_run_res, nt, nf, 0)

In [ ]:
optim_plot(fixedmu=fixedmu_run_res, tail=0.01)

In [ ]:
plt.hist(global_run_res[1][np.argsort(global_run_res[0])[:10000]][:, 1])

In [ ]:
best = thetas[np.aargsort(nlls)[200]]

N = 5_000_000

#np.random.seed(1)
worker = partial(nlopt_worker, nt=nt, nf=nf) 
global_run_res = run_nloptims(worker, partial(random_start_simplex, nt=nt, nf=nf), N)

In [ ]:
optim_plot(global_run=global_run_res, tail=0.001)

In [ ]:
best_theta = nloptim_summary(global_run_res, nt, nf) # 3rd run

In [ ]:
best_theta = nloptim_summary(global_run_res, nt, nf) # 2nd run

In [ ]:
best_theta = nloptim_summary(global_run_res, nt, nf) # first run

In [ ]:
bt = np.array([0.00195829561471389, 1.232386727173979e-08, 0.574 , 0.751,  0.906,
          0.002 , 0.098,  0.074, 0.044,  0.029,  0.001, 0   ,   0.011 , 0.012,
          0.381,  0.11,   0.008])
          

### Experimental Perturbation Starts

It could be an issue of choosing the right perturbation scale, but this looked like it didn't work. I think that this is more becausing finding the true MLE in this space is hard — there are lots of local optima that need to be explored.

In [ ]:
def trunc_normal(loc, scale, lower, upper):
    a, b = (lower - loc) / scale, (upper - loc) / scale
    return np.random.normal(loc, scale)
    return truncnorm.rvs(a, b)

def perturber(theta, log10_pi0_bounds=(-4, -3),
            log10_mu_bounds=(-11, -7), pi0_scale=1e-6, 
            mu_scale=1e-12, dfe_scale=1e-04):
    def func():
        new_theta = np.copy(theta)
        normal = truncnorm.rvs

        new_theta[0] = trunc_normal(theta[0], pi0_scale, 10**log10_pi0_bounds[0], 10**log10_pi0_bounds[1])
        new_theta[1] = trunc_normal(theta[1], mu_scale, 10**log10_mu_bounds[0], 10**log10_mu_bounds[1])
        for i in range(2, theta.size):
            w = theta[i]
            new_theta[i] = trunc_normal(w, dfe_scale, 10**log10_mu_bounds[0]/w, 10**log10_mu_bounds[1]/w)
        return new_theta
    return func

In [ ]:
perturb_starts = perturber(best_theta)
run_res = run_nloptims(worker, perturb_starts, 4000)

In [ ]:
optim_plot(test=run_res, tail=0.6)

## Penalized Methods

Some of the instability of convergence to the optimum is that the model cannot choose between weak or strong mutation (and visa versa for selection). 

We could put a prior on this mutation rate, since we have many other estimates. A recent paper (Tian, Browning, and Browning, 2019) estimated the mutation rate as $\mu = 1.28 \times 10^{-8}$ ($95\%$ CI, $1.02 \times 10^{-8}, 1.56 \times 10^{-8}$).

In [ ]:
sigma

In [ ]:
mean = 1.28e-8
ci = [1.02e-8, 1.56e-8]

def ci2params(mean, ci, alpha=0.05):
    crit = stats.norm.ppf(1 - alpha/2)
    sigma1 = (mean-ci[0])/crit
    sigma2 = (ci[1]-mean)/crit
    return mean, np.mean((sigma1, sigma2))

mean, sigma = ci2params(mean, ci)

z = np.linspace(mean - 2.5*sigma, mean + 2.5*sigma, 1000)
y = stats.norm(mean, sigma).pdf(z)
plt.plot(z, y)
#plt.hist(stats.norm(mean, sigma).rvs(10000), 100, density=True)
#plt.axvline(mean -sigma)
plt.axvline(ci[0])
plt.axvline(ci[1])

In [ ]:
r = 1/(sigma / 1e3)**2

r, -r * (mean - 4.6e-9)**2 / 2 / 1e11

In [ ]:
(mean - 4.6e-9)**2

In [ ]:
negll_c(best_theta, Y, B, w), penalized_negll_c(best_theta, Y, B, w, 1.23e-8, r = r)

In [ ]:
def penalized_negll_nlopt(Y, B, w, mu0, r):
    # NLOPT needs to accept a grad argument
    def func(x, grad):
        return penalized_negll_c(x, Y, B, w, mu0, r)
    return func


def penalized_nlopt_worker(start, nt, nf, mu0, r):
    nparams = nt*nf + 2
    opt = nlopt.opt(nlopt.LN_COBYLA, nparams)
    #opt = nlopt.opt(nlopt.LN_BOBYQA, nparams)    
    #opt = nlopt.opt(nlopt.AUGLAG, nparams)
    #opt.set_local_optimizer(nlopt.LN_COBYLA)
    nll = penalized_negll_nlopt(Y, Bp, w, mu0=mu0, r=r)
    opt.set_min_objective(nll) 
    hl, hu = inequality_constraint_functions(nt, nf)
    tols = np.repeat(1e-11, nf)
    opt.add_inequality_mconstraint(hl, tols)
    opt.add_inequality_mconstraint(hu, tols)
    ce = equality_constraint_function(nt, nf)
    opt.add_equality_mconstraint(ce, tols)
    lb, ub = bounds_simplex(nt, nf)
    opt.set_lower_bounds(lb)
    opt.set_upper_bounds(ub)
    opt.set_xtol_abs(np.array([1e-4, 1e-10] + [1e-4] * nt*nf))
    #opt.set_ftol_abs(1e8)
    #opt.set_stopval(923543002497)
    opt.set_maxeval(100000)
    x = random_start_simplex(nt, nf)
    assert x.size == nparams
    mle = opt.optimize(x)
    nll = opt.last_optimum_value()
    success = opt.last_optimize_result()
    return nll, mle, success

N = 10_000

np.random.seed(1)
worker = partial(penalized_nlopt_worker, nt=nt, nf=nf, mu0=1.23e-8, r=0) 
#penalized_run_res = run_nloptims(worker, partial(random_start_simplex, nt=nt, nf=nf), N)

## Scratch

In [ ]:

def normal_negll_freemut_nlopt(Y, B, w):
    def func(x, grad):
        new_theta = np.zeros(x.size + 1)
        new_theta[0] = x[0]
        # fix mutation rate to one and let W represent mutation rates to various classes
        new_theta[1] = 1.
        new_theta[2:] = x[1:] # times mutation rates        
        return -normal_ll_c(new_theta, Y, B, w)
    return func

In [ ]:
worker(random_start_simplex(nt, nf))

In [ ]:
nloptim_summary(penalized_run_res, nt, nf)

In [ ]:
optim_plot(penalized_run=penalized_run_res)

In [ ]:
optim_plot(penalized_run=penalized_run_res, tail=0.01)

In [ ]:
pbr = nloptim_summary(penalized_run_res, nt, nf)

In [ ]:
pbr = nloptim_summary(penalized_run_res, nt, nf)

In [ ]:
nloptim_summary(penalized_run_res, nt, nf)

## LIPO Optimization

In [ ]:
from lipo import GlobalOptimizer

In [ ]:
params = ['pi0'] + [f"w{i}" for i in range(nt*nf)]

In [ ]:
glb, gub = [dict(zip(params, x.tolist())) for x in bounds_mutation(nf, nt)]

In [ ]:
glb

In [ ]:
for l in [f"w{i}" for i in range(nt*nf)]:
    print(l + ", ", end='')

In [ ]:
def negll_freemut_alt(Y, B, w):
    def func(pi0, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14):
        new_theta = np.zeros(17)
        new_theta[0] = pi0
        # fix mutation rate to one and let W represent mutation rates to various classes
        new_theta[1] = 1.
        new_theta[2:] = [w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14] # times mutation rates        
        return negll_c(new_theta, Y, B, w)
    return func

nllBp = negll_freemut_alt(Y, Bp, w)
nllB = negll_freemut_alt(Y, B, w)

In [ ]:
evaluations

In [ ]:
pre_eval_x = dict(zip(params, random_start_mutation(nt, nf).tolist()))
evaluations = [(pre_eval_x, nllBp(**pre_eval_x))]

search = GlobalOptimizer(
    nllBp,
    lower_bounds=glb,
    upper_bounds=gub,
    #categories={"z": ["a", "b"]},
    evaluations=evaluations,
    maximize=False,
)

num_function_calls = 10000
search.run(num_function_calls)

In [ ]:
search.optimum

## Likelihood Diagnostics

In [ ]:
Y

In [ ]:
plt.plot(np.arange(Y.shape[0]), Y[:, 1])

In [ ]:
from bgspy.likelihood import predict_simplex
pi = predict_simplex(best_theta, B, w)

N = Y.sum(axis=1)

x = Y[:, 0] / (Y.sum(axis=1))
plt.plot(stats.norm(loc=N*pi, scale=N*pi*(1-pi)).logpdf(x))
#plt.ylim(-0.2e2, 0)

In [ ]:
from bgspy.likelihood import predict_simplex
pi = predict_simplex(best_theta, B, w)

N = Y.sum(axis=1)

x = Y[:, 1] / (Y.sum(axis=1))
y = stats.norm(loc=N*pi, scale=N*pi*(1-pi)).logpdf(x)
plt.plot(y)
plt.ylim(-0.2e2, 0)

In [ ]:
_ = plt.hist(x, 200, density=True)

In [ ]:
_ = plt.hist(x, 100, density=True)
z = np.linspace(0.0001, 0.008, 100)
# trimmed mean
alpha = 0.05
idx = (np.nanquantile(x, alpha/2) < x) & (np.nanquantile(x, 1-alpha/2) > x)
xt = x[idx]
plt.plot(z, stats.norm(loc=np.nanmean(xt), scale=np.nanstd(xt)).pdf(z), c='r')
plt.plot(z, stats.binom(p=np.nanmean(xt), n=N.mean()).pmf(z), c='g')

In [ ]:
N.mean()

In [ ]:
xx = x[~np.isnan(x)]
plt.plot(standardize(xx))

In [ ]:
import seaborn as sns
sns.kdeplot(x=xx[:-1:1], y=xx[1::1], fill=True,levels=100, cmap="mako")
plt.ylim(0.0005, 0.003)
plt.xlim(0.0005, 0.003)

In [ ]:
def standardize(x):
    return (x - np.nanmean(x)) / (np.nanstd(x))

_ = plt.hist(standardize(x), 100, density=True)
#z = np.linspace(0.0001, 0.008, 100)
# trimmed mean
#xt = x[(np.nanquantile(x, 0.01) < x) & (np.nanquantile(x, 0.99) > x)]
#plt.plot(z, stats.norm(loc=np.nanmean(xt), scale=np.nanstd(xt)).pdf(z), c='r')

In [ ]:
import scipy

In [ ]:

# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * ~(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    # Compute posterior mean
    μ2 = solved @ y1
    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    return μ2, Σ2  # mean, covariance

In [ ]:
S11 = exponentiated_quadratic(yy[:, None], yy[:, None])

In [ ]:
S11.sum()

In [ ]:
X1 = X2 = np.arange(yy.size)[:, None]
y1 = yy

X2 = np.random.uniform(0, yy.size, 1000)[:, None]
mean, Sigma = GP(X1, y1, X2, exponentiated_quadratic)

In [ ]:
y2 = np.random.multivariate_normal(mean=mean, cov=Sigma, size=yy.size)


In [ ]:
yy.size

In [ ]:
y2.shape

In [ ]:
plt.plot(y2[0, :])
plt.plot(y2[10, :], linestyle='dashed')
plt.ylim(-0.2e2, -10)
#plt.xlim(1000, 1500)

In [ ]:
from jax import jacfwd, jacrev
import jax.numpy as jnp
#from jax.config import config
#config.update("jax_enable_x64", True)

In [ ]:
def ll_jax(theta, Y, logB, w):
    nS = Y[:, 0]
    nD = Y[:, 1]
    nx, nw, nt, nf = logB.shape
    # mut weight params
    pi0, mu, W = theta[0], theta[1], theta[2:]
    W = W.reshape((nt, nf))
    # interpolate B(w)'s
    logBw = jnp.zeros(nx, dtype=jnp.float32)
    for i in range(nx):
        for j in range(nt):
            for k in range(nf):
                logBw = logBw.at[i].add(jnp.interp(mu*W[j, k], w, logB[i, :, j, k]))
    log_pibar = jnp.log(pi0) + logBw
    llm = nD*log_pibar + nS*jnp.log1p(-jnp.exp(log_pibar))
    return jnp.sum(llm)

def reduce_theta(theta, nt, nf):
    pi0, mu, W = theta[0], theta[1], theta[2:]
    W = W.reshape((nt, nf))
    new_theta = np.empty((nt-1)*nf + 2)
    new_theta[0] = theta[0]
    new_theta[1] = theta[1]
    new_theta[2:] = W[1:, :].flat
    return new_theta

In [ ]:
reduce_theta(best_theta, nt, nf)[2:].reshape((nt-1, nf))

In [ ]:
func = partial(ll_jax, Y=Y, logB=B, w=w)

In [ ]:
pbr[2:].reshape((nt, nf))

In [ ]:
def hessian(f):
    return jacfwd(jacrev(f))
H = hessian(func)(pbr)

In [ ]:
np.linalg.inv(H)

In [ ]:
np.linalg.det(H)

In [ ]:
np.diag(H), np.trace(H)

In [ ]:
np.linalg.eigvals(H)

In [ ]:
H

In [ ]:
h = np.copy(H)

In [ ]:
h[1,1] = np.nan
plt.imshow(h[2:, 2:])

In [ ]:
h

In [ ]:
best_theta

In [ ]:
H[0, 0]

In [ ]:
np.linalg.det(H[:2, :2])

In [ ]:
H

In [ ]:
np.linalg.inv(H)